In [13]:
pip install "numpy<2.0"  


   ---------------------------------------- 0.0/15.5 MB ? eta -:--:--
   --- ------------------------------------ 1.3/15.5 MB 7.4 MB/s eta 0:00:02
   -------- ------------------------------- 3.4/15.5 MB 8.8 MB/s eta 0:00:02
   -------------------- ------------------- 8.1/15.5 MB 13.6 MB/s eta 0:00:01
   ---------------------------------------  15.2/15.5 MB 19.1 MB/s eta 0:00:01
   ---------------------------------------- 15.5/15.5 MB 18.8 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 30.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.5 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.


In [ ]:
pip install tqdm

In [ ]:
pip install seqeval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 298.3 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16161 sha256=8e8c32ed191bf5767f078bc91b41c063c5770561193dc26c20c2f5bccda34f83
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval


In [ ]:
pip install pytorch-crf


In [32]:
pip install seaborn

Note: you may need to restart the kernel to use updated packages.


In [1]:
pip install nltk

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install gensim

   ---------------------------------------- 0.0/24.0 MB ? eta -:--:--
   -- ------------------------------------- 1.3/24.0 MB 6.7 MB/s eta 0:00:04
   ------- -------------------------------- 4.7/24.0 MB 11.9 MB/s eta 0:00:02
   ------------- -------------------------- 8.4/24.0 MB 13.7 MB/s eta 0:00:02
   ---------------------- ----------------- 13.4/24.0 MB 16.4 MB/s eta 0:00:01
   ------------------------------------ --- 21.8/24.0 MB 21.5 MB/s eta 0:00:01
   ---------------------------------------- 24.0/24.0 MB 21.7 MB/s eta 0:00:00
   ---------------------------------------- 0.0/45.9 MB ? eta -:--:--
   ---------- ----------------------------- 11.8/45.9 MB 52.9 MB/s eta 0:00:01
   ------------- -------------------------- 15.7/45.9 MB 58.4 MB/s eta 0:00:01
   ------------------- -------------------- 22.0/45.9 MB 35.7 MB/s eta 0:00:01
   -------------------------------- ------- 37.7/45.9 MB 46.2 MB/s eta 0:00:01
   ---------------------------------------- 45.9/45.9 MB 46.4 MB/s eta 0:0

  You can safely remove it manually.
  You can safely remove it manually.


In [29]:
import torch
import torch.nn as nn
from torchcrf import CRF
from transformers import AutoModel, AutoTokenizer
from datasets import load_dataset
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
from functools import partial
import uuid
from collections import Counter
from seqeval.metrics import classification_report
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix
import numpy as np

import logging
import re


In [50]:
import logging

In [30]:
import re

In [15]:
class BiLSTMCRFForTokenClassification(nn.Module):
    def __init__(self, embedding_dim, hidden_dim, num_labels, dropout=0.1):
        super(BiLSTMCRFForTokenClassification, self).__init__()
        self.embedding_dim = embedding_dim
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, bidirectional=True, batch_first=True)
        self.dropout = nn.Dropout(dropout)
        self.fc = nn.Linear(hidden_dim * 2, num_labels)
        self.crf = CRF(num_labels, batch_first=True)

    def forward(self, embeddings, attention_mask=None, labels=None):
        lstm_out, _ = self.lstm(embeddings)
        lstm_out = self.dropout(lstm_out)
        emissions = self.fc(lstm_out)

        mask = attention_mask.bool()

        if labels is not None:
            labels = torch.where(labels == -100, 0, labels)
            log_likelihood = self.crf(emissions, labels, mask=mask)
            return -log_likelihood
        else:
            prediction = self.crf.decode(emissions, mask=mask)
            return prediction

In [16]:
class PIIDataset(Dataset):
    def __init__(self, examples, tokenizer, bert_model, max_length, device):
        self.examples = examples
        self.tokenizer = tokenizer
        self.bert_model = bert_model
        self.max_length = max_length
        self.device = device

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, idx):
        item = self.examples[idx]
        encoding = self.tokenizer(item['source_text'], truncation=True, padding='max_length', max_length=self.max_length, return_tensors='pt')
        input_ids = encoding['input_ids'].squeeze().to(self.device)
        attention_mask = encoding['attention_mask'].squeeze().to(self.device)
    
        with torch.no_grad():
            embeddings = self.bert_model(input_ids.unsqueeze(0), attention_mask.unsqueeze(0)).last_hidden_state.squeeze(0)
    
        labels = torch.tensor(item['labels'], dtype=torch.long).to(self.device)
    
        return {
            'embeddings': embeddings,
            'attention_mask': attention_mask,
            'labels': labels,
            'input_ids': input_ids  # Added this line to include input_ids in the returned dictionary
        }

In [19]:
def process_data(example, labels2int, tokenizer, max_length):
    text, labels = example["source_text"], example["privacy_mask"]

    encoding = tokenizer(text, truncation=True, padding='max_length', max_length=max_length,
                         return_offsets_mapping=True, return_special_tokens_mask=True)
    label_ids = [labels2int["O"]] * len(encoding['input_ids'])

    for label in labels:
        start_char, end_char = label['start'], label['end']
        token_start = encoding.char_to_token(start_char)
        token_end = encoding.char_to_token(end_char - 1)

        if token_start is not None and token_end is not None:
            label_type = label['label']
            if label_type != "CARDISSUER":
                if label_type.startswith("GIVENNAME"):
                    label_type = "GIVENNAME" 
                elif label_type.startswith("LASTNAME"):
                    label_type = "LASTNAME"

                label_ids[token_start] = labels2int.get(f'B-{label_type}', labels2int["O"])
                for i in range(token_start + 1, token_end + 1):
                    label_ids[i] = labels2int.get(f'I-{label_type}', labels2int["O"])

    return {
        'input_ids': encoding['input_ids'],
        'attention_mask': encoding['attention_mask'], 
        'labels': label_ids
    }

In [20]:
def train(model, train_loader, val_loader, optimizer, device, num_epochs, id2label, patience=3):
    train_losses = []
    val_losses = []
    best_val_loss = float('inf')
    epochs_no_improve = 0
    best_model = None

    for epoch in range(num_epochs):
        model.train()
        total_loss = 0
        progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}")
        for batch in progress_bar:
            input_ids = batch['input_ids'].to(device)
            embeddings = batch['embeddings'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)
            labels = torch.where(labels == -100, 0, labels)
            
            optimizer.zero_grad()
            loss = model(embeddings, attention_mask=attention_mask, labels=labels)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
            progress_bar.set_postfix({'train_loss': f"{loss.item():.4f}"})
        
        avg_train_loss = total_loss / len(train_loader)
        train_losses.append(avg_train_loss)
        print(f"\nEpoch {epoch+1}/{num_epochs}, Average training loss: {avg_train_loss:.4f}")
        
        model.eval()
        val_loss = 0
        all_preds = []
        all_labels = []
        with torch.no_grad():
            for batch in val_loader:
                input_ids = batch['input_ids'].to(device)
                embeddings = batch['embeddings'].to(device)
                attention_mask = batch['attention_mask'].to(device)
                labels = batch['labels'].to(device)
                labels = torch.where(labels == -100, 0, labels)
                
                loss = model(embeddings, attention_mask=attention_mask, labels=labels)
                val_loss += loss.item()
                preds = model(embeddings, attention_mask=attention_mask)
                
                for i, mask in enumerate(attention_mask):
                    pred = [id2label[p] for p, m in zip(preds[i], mask) if m.item() == 1]
                    true = [id2label[l.item()] for l, m in zip(labels[i], mask) if m.item() == 1]
                    if pred and true:
                        all_preds.append(pred)
                        all_labels.append(true)
        
        avg_val_loss = val_loss / len(val_loader)
        val_losses.append(avg_val_loss)
        print(f"Validation loss: {avg_val_loss:.4f}")
        
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            epochs_no_improve = 0
            best_model = model.state_dict()
            torch.save(best_model, f"best_model_{str(uuid.uuid4())}.pth")
        else:
            epochs_no_improve += 1
            if epochs_no_improve == patience:
                print(f"Early stopping triggered after epoch {epoch+1}")
                model.load_state_dict(best_model)
                break
        
        print("Label distribution in true labels:")
        print(Counter(sum(all_labels, [])))
        print("\nLabel distribution in predictions:")  
        print(Counter(sum(all_preds, [])))
        print(f"\nNumber of sequences evaluated: {len(all_labels)}")
        print(f"Number of sequences with predictions: {len(all_preds)}") 
        print(f"Number of sequences with labels: {len(all_labels)}")
        print("\nClassification Report:")
        print(classification_report(all_labels, all_preds))
    
    # Plot learning curve
    plt.figure(figsize=(10, 6))
    plt.plot(range(1, len(train_losses) + 1), train_losses, label='Training Loss')
    plt.plot(range(1, len(val_losses) + 1), val_losses, label='Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.title('Learning Curve')
    plt.legend()
    plt.savefig('learning_curve.png')
    plt.close()
    
    return model, all_labels, all_preds

In [21]:
def plot_confusion_matrix(true_labels, pred_labels, label_list):
    # Flatten the lists of labels
    true_flat = [item for sublist in true_labels for item in sublist]
    pred_flat = [item for sublist in pred_labels for item in sublist]

    # Create confusion matrix
    cm = confusion_matrix(true_flat, pred_flat, labels=label_list)

    # Plot confusion matrix
    plt.figure(figsize=(20, 16))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=label_list, yticklabels=label_list)
    plt.title('Confusion Matrix')
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.xticks(rotation=90)
    plt.yticks(rotation=0)
    plt.tight_layout()
    plt.savefig('confusion_matrix.png')
    plt.close()


In [38]:
def main():
    # Define labels (excluding CARDISSUER)
    labels = [
        "BOD", "BUILDING", "CITY", "COUNTRY", "DATE", "DRIVERLICENSE",
        "EMAIL", "GEOCOORD", "GIVENNAME", "IDCARD", "IP", "LASTNAME",
        "PASS", "PASSPORT", "POSTCODE", "SECADDRESS", "SEX",
        "SOCIALNUMBER", "STATE", "STREET", "TEL", "TIME", "TITLE", "USERNAME"
    ]

    label_list = ["O"] + [f"B-{label}" for label in labels] + [f"I-{label}" for label in labels]
    labels = [f"I-{label}" for label in labels] + [f"B-{label}" for label in labels] + ["O"]
    label2id = {label: i for i, label in enumerate(labels)}
    id2label = {v: k for k, v in label2id.items()}

    # Load tokenizer and BERT model
    bert_model_name = "bert-base-cased"
    tokenizer = AutoTokenizer.from_pretrained(bert_model_name)
    bert_model = AutoModel.from_pretrained(bert_model_name)
    bert_model.eval()  # Set BERT to evaluation mode
    for param in bert_model.parameters():
        param.requires_grad = False  # Freeze BERT parameters

    # Load and process dataset
    ds = load_dataset("ai4privacy/pii-masking-300k")
    ds = ds.filter(lambda x: x["language"] == "English", num_proc=4)

    # Remove rows with "CARDISSUER" label
    ds = ds.filter(lambda x: "CARDISSUER" not in x["mbert_bio_labels"], num_proc=4)

    max_length = 512
    ds = ds.map(
        partial(process_data, labels2int=label2id, tokenizer=tokenizer, max_length=max_length),
        remove_columns=['privacy_mask', 'span_labels', 'mbert_text_tokens', 'mbert_bio_labels', 'id', 'language', 'set'],
        num_proc=8
    )

    # Split the validation set into validation and test sets
    val_test_split = 0.3
    val_dataset = ds["validation"]
    val_dataset = val_dataset.shuffle(seed=42)
    val_size = int(len(val_dataset) * (1 - val_test_split))
    test_size = len(val_dataset) - val_size
    val_dataset, test_dataset = val_dataset.select(range(val_size)), val_dataset.select(range(val_size, len(val_dataset)))

    # Define device
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")

    # Move BERT model to device
    bert_model = bert_model.to(device)

    # Create datasets and dataloaders
    train_dataset = PIIDataset(ds["train"], tokenizer, bert_model, max_length, device)
    val_dataset = PIIDataset(val_dataset, tokenizer, bert_model, max_length, device)
    test_dataset = PIIDataset(test_dataset, tokenizer, bert_model, max_length, device)

    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=64)
    test_loader = DataLoader(test_dataset, batch_size=64)
    
    # Initialize model
    model = BiLSTMCRFForTokenClassification(
        bert_model.config.hidden_size,
        hidden_dim=128,
        num_labels=len(labels)
    ).to(device)

    # Define optimizer
    optimizer = torch.optim.Adam(model.parameters(), lr=2e-5)

    # Train the model
    num_epochs = 29
    print(f"Starting training for {num_epochs} epochs...")
    model, true_labels, pred_labels = train(model, train_loader, val_loader, optimizer, device, num_epochs, id2label)

    # Save the model
    model_save_path = f"bilstm_crf_pii_model_bert_{str(uuid.uuid4())}.pth"
    torch.save(model.state_dict(), model_save_path)
    print(f"Model saved successfully to {model_save_path}")

     # Test the model
    model.eval()
    test_preds = []
    test_labels = []
    with torch.no_grad():
        for batch in tqdm(test_loader, desc="Testing"):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)
    
            # Generate embeddings using the BERT model
            embeddings = bert_model(input_ids, attention_mask=attention_mask).last_hidden_state
    
            # Pass embeddings to the model instead of input_ids
            preds = model(embeddings, attention_mask=attention_mask)
    
            for i, mask in enumerate(attention_mask):
                pred = [id2label[p] for p, m in zip(preds[i], mask) if m.item() == 1]
                true = [id2label[l.item()] for l, m in zip(labels[i], mask) if m.item() == 1]
                if pred and true:
                    test_preds.append(pred)
                    test_labels.append(true)

    # Print test results
    print("\nTest Results:")
    print(classification_report(test_labels, test_preds))

    # Plot confusion matrix for test results
    plot_confusion_matrix(test_labels, test_preds, label_list)

In [39]:
if __name__ == "__main__":
    main()

C:\Users\harsh\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Map (num_proc=8):   0%|          | 0/29908 [00:00<?, ? examples/s]

Map (num_proc=8):   0%|          | 0/7946 [00:00<?, ? examples/s]

Using device: cuda
Starting training for 29 epochs...


Epoch 1/29: 100%|██████████████████████████████████████████████| 935/935 [14:52<00:00,  1.05it/s, train_loss=2269.7002]


...


Epoch 28/29, Average training loss: 253.9501
Validation loss: 580.8769
Label distribution in true labels:
Counter({'O': 531915, 'I-IP': 27562, 'I-EMAIL': 17591, 'I-DRIVERLICENSE': 13938, 'I-USERNAME': 11097, 'I-IDCARD': 10771, 'I-TEL': 10585, 'I-SOCIALNUMBER': 10568, 'I-PASSPORT': 7842, 'I-PASS': 7721, 'I-BOD': 6833, 'I-TIME': 5614, 'I-DATE': 5435, 'I-LASTNAME': 4877, 'I-POSTCODE': 4194, 'I-STREET': 3465, 'I-GIVENNAME': 3225, 'I-CITY': 2637, 'B-TIME': 2618, 'B-LASTNAME': 2240, 'B-USERNAME': 2023, 'B-EMAIL': 1846, 'B-IDCARD': 1825, 'B-GIVENNAME': 1824, 'B-SOCIALNUMBER': 1724, 'B-PASSPORT': 1696, 'B-DRIVERLICENSE': 1693, 'B-BOD': 1656, 'I-GEOCOORD': 1621, 'B-IP': 1527, 'I-SEX': 1493, 'B-SEX': 1437, 'B-CITY': 1410, 'B-STATE': 1398, 'B-TITLE': 1361, 'B-BUILDING': 1356, 'B-TEL': 1342, 'B-POSTCODE': 1326, 'B-STREET': 1326, 'I-SECADDRESS': 1269, 'B-DATE': 1213, 'B-PASS': 1166, 'B-COUNTRY': 1124, 'I-STATE': 1042, 'I-BUILDING': 860, 'I-TITLE': 822, 'B-SECADDRESS': 594, 'I-COUNTRY': 590, 'B-GEO

Epoch 29/29: 100%|███████████████████████████████████████████████| 935/935 [14:54<00:00,  1.04it/s, train_loss=90.3371]



Epoch 29/29, Average training loss: 246.9887
Validation loss: 572.8506
Label distribution in true labels:
Counter({'O': 531915, 'I-IP': 27562, 'I-EMAIL': 17591, 'I-DRIVERLICENSE': 13938, 'I-USERNAME': 11097, 'I-IDCARD': 10771, 'I-TEL': 10585, 'I-SOCIALNUMBER': 10568, 'I-PASSPORT': 7842, 'I-PASS': 7721, 'I-BOD': 6833, 'I-TIME': 5614, 'I-DATE': 5435, 'I-LASTNAME': 4877, 'I-POSTCODE': 4194, 'I-STREET': 3465, 'I-GIVENNAME': 3225, 'I-CITY': 2637, 'B-TIME': 2618, 'B-LASTNAME': 2240, 'B-USERNAME': 2023, 'B-EMAIL': 1846, 'B-IDCARD': 1825, 'B-GIVENNAME': 1824, 'B-SOCIALNUMBER': 1724, 'B-PASSPORT': 1696, 'B-DRIVERLICENSE': 1693, 'B-BOD': 1656, 'I-GEOCOORD': 1621, 'B-IP': 1527, 'I-SEX': 1493, 'B-SEX': 1437, 'B-CITY': 1410, 'B-STATE': 1398, 'B-TITLE': 1361, 'B-BUILDING': 1356, 'B-TEL': 1342, 'B-POSTCODE': 1326, 'B-STREET': 1326, 'I-SECADDRESS': 1269, 'B-DATE': 1213, 'B-PASS': 1166, 'B-COUNTRY': 1124, 'I-STATE': 1042, 'I-BUILDING': 860, 'I-TITLE': 822, 'B-SECADDRESS': 594, 'I-COUNTRY': 590, 'B-GEO

Testing: 100%|█████████████████████████████████████████████████████████████████████████| 38/38 [01:47<00:00,  2.83s/it]



Test Results:
               precision    recall  f1-score   support

          BOD       0.95      0.94      0.95       660
     BUILDING       0.95      0.95      0.95       586
         CITY       0.91      0.94      0.93       606
      COUNTRY       0.93      0.95      0.94       441
         DATE       0.87      0.89      0.88       487
DRIVERLICENSE       0.85      0.85      0.85       723
        EMAIL       0.96      0.98      0.97       765
     GEOCOORD       0.80      0.83      0.82        64
    GIVENNAME       0.72      0.75      0.74       734
       IDCARD       0.86      0.87      0.86       831
           IP       0.92      0.95      0.93       636
     LASTNAME       0.78      0.77      0.78      1007
         PASS       0.73      0.76      0.75       442
     PASSPORT       0.82      0.86      0.84       728
     POSTCODE       0.91      0.95      0.93       578
   SECADDRESS       0.94      0.92      0.93       264
          SEX       0.94      0.91      0.92     